## Workspace

In [14]:
from azureml.core import Workspace

ws = Workspace.from_config()

print(ws)

Workspace.create(name='yuxinazml', subscription_id='4212cdcd-1336-4578-8079-a10d6f3df25e', resource_group='azuremlgroup')


## Environment

In [15]:
from azureml.core import Environment

sklearn_env = Environment.get(workspace=ws, name='AzureML-Tutorial')

## Train my poly regression model on instance.

In [16]:
import joblib 
import pandas as pd

import sklearn
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge

# Import data munging tools
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures

# Import error metric
from sklearn.metrics import mean_absolute_error

data = pd.read_csv('./people_in_supermarket_v2.csv')
X = data.iloc[:, 0:-1]
y = data['numOfpeopleNextmin']

x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

# redige regression
poly_features = PolynomialFeatures(degree = 11) 
# Transform training and test datasets 
x_train_poly = poly_features.fit_transform(x_train)
x_test_poly = poly_features.fit_transform(x_test)

lambda_value = 1E-15
ridge_reg = Ridge(alpha = lambda_value, normalize = True)
# Fit the model
ridge_reg.fit(x_train_poly,y_train)

joblib.dump(ridge_reg, "0073_poly_regression_model.pkl") # store the trained model in pkl file 

['0073_poly_regression_model.pkl']

In [17]:
# Compute the predictions of the model for test set
y_reg_test_pred = ridge_reg.predict(x_test_poly)
# Compute the Mean Absoluate Error for test set, then append the value to the lists 
abses_test_error = mean_absolute_error(y_test, y_reg_test_pred)

print(abses_test_error) 

2.9492153069114933


## Register model

In [31]:
from azureml.core import Model
from azureml.core.resource_configuration import ResourceConfiguration

#-------------------my poly LR model----------------------
model = Model.register(model_path="0073_poly_regression_model.pkl",
                       model_name="0073_poly_regression_model",
                       description="0073 Poly ridge regression model",
                       workspace=ws)

Registering model 0073_poly_regression_model


## Deployment

Create environment

In [40]:
import sklearn

from azureml.core.environment import Environment

environment = Environment("LocalDeploy")
environment.python.conda_dependencies.add_pip_package("inference-schema[numpy-support]")
environment.python.conda_dependencies.add_pip_package("joblib")
environment.python.conda_dependencies.add_pip_package("scikit-learn=={}".format(sklearn.__version__))

Configure

In [41]:
# Define an inference configuration
from azureml.core import Environment
from azureml.core.model import InferenceConfig

# -------------------Inference config--------------------------
inference_config = InferenceConfig(
    environment = environment,
    entry_script="score.py"
)

# -------------------Local Deploy config--------------------------
from azureml.core.webservice import LocalWebservice

deployment_config = LocalWebservice.deploy_configuration()

# -------------------WEB Deploy config--------------------------
from azureml.core.webservice import AciWebservice

web_deployment_config = AciWebservice.deploy_configuration(
    cpu_cores = 0.5, memory_gb = 1, auth_enabled = False
)

Local Deploy

In [43]:
# deploy
local_service = Model.deploy(
    ws, 
    "mscproj-poly-regression-model", 
    [model], 
    inference_config,
    deployment_config,
    overwrite = True 
)

local_service.wait_for_deployment(show_output=True)

Generating Docker build context.
Package creation Succeeded
Logging into Docker registry 71ab244ead974779bb6a9f390be30da0.azurecr.io
Logging into Docker registry 71ab244ead974779bb6a9f390be30da0.azurecr.io
Building Docker image from Dockerfile...
Step 1/5 : FROM 71ab244ead974779bb6a9f390be30da0.azurecr.io/azureml/azureml_299b69c7fbf0f4df0b504c4c5025191b
 ---> b3d9beafdfc6
Step 2/5 : COPY azureml-app /var/azureml-app
 ---> af6e25507e49
Step 3/5 : RUN mkdir -p '/var/azureml-app' && echo eyJhY2NvdW50Q29udGV4dCI6eyJzdWJzY3JpcHRpb25JZCI6IjQyMTJjZGNkLTEzMzYtNDU3OC04MDc5LWExMGQ2ZjNkZjI1ZSIsInJlc291cmNlR3JvdXBOYW1lIjoiYXp1cmVtbGdyb3VwIiwiYWNjb3VudE5hbWUiOiJ5dXhpbmF6bWwiLCJ3b3Jrc3BhY2VJZCI6IjcxYWIyNDRlLWFkOTctNDc3OS1iYjZhLTlmMzkwYmUzMGRhMCJ9LCJtb2RlbHMiOnt9LCJtb2RlbHNJbmZvIjp7fX0= | base64 --decode > /var/azureml-app/model_config_map.json
 ---> Running in a584f2dcde34
 ---> d599e5439517
Step 4/5 : RUN mv '/var/azureml-app/tmpg21znf_w.py' /var/azureml-app/main.py
 ---> Running in f294e59829eb
 -

In [44]:
import requests
import json

uri = local_service.scoring_uri
print(uri)
headers = {"Content-Type": "application/json"}
data = {
    "data": [[5, 14, 32, 60]] 
}
data_json = json.dumps(data)
response = requests.post(uri, data = data_json, headers = headers)
print(response.json()[0])


http://localhost:49175/score
33.23912437929539


In [45]:
# test by model traind by instance
x_poly = poly_features.fit_transform([[5, 14, 32, 60]]) 
print(ridge_reg.predict(x_poly)) 

[33.23912438]


Web Deploy

In [46]:
# WEB deploy
web_service = Model.deploy(
    ws,
    "mscproj-ml-webservice",
    [model],
    inference_config,
    web_deployment_config,
    overwrite = True,
)
web_service.wait_for_deployment(show_output=True)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2021-09-09 13:56:07+00:00 Creating Container Registry if not exists.
2021-09-09 13:56:07+00:00 Registering the environment.
2021-09-09 13:56:07+00:00 Use the existing image.
2021-09-09 13:56:07+00:00 Generating deployment configuration.
2021-09-09 13:56:08+00:00 Submitting deployment to compute.
2021-09-09 13:56:14+00:00 Checking the status of deployment mscproj-ml-webservice..
2021-09-09 13:58:15+00:00 Checking the status of inference endpoint mscproj-ml-webservice.
Succeeded
ACI service creation operation finished, operation "Succeeded"


WEB update

In [ ]:
# web_service.update(
#     models = [model], 
#     inference_config = inference_config,
#     deployment_config = web_deployment_config
#     )

WEB service test

In [47]:
print(f'URL: {web_service.scoring_uri}') 

URL: http://be61aad3-0d12-44d3-9cd0-04ebed5dfeeb.uksouth.azurecontainer.io/score


In [48]:
import requests
import json

uri = web_service.scoring_uri
print(uri)
headers = {"Content-Type": "application/json"}
data = {
   "data": [[4, 12, 17, 30]] 
}
data_json = json.dumps(data)
response = requests.post(uri, data = data_json, headers = headers)
print(response.json())


http://be61aad3-0d12-44d3-9cd0-04ebed5dfeeb.uksouth.azurecontainer.io/score
[25.7861097825662]


In [ ]:
x_poly = poly_features.fit_transform([[4, 12, 17, 30]]) 
print(ridge_reg.predict(x_poly)) 